# Variational Deep Q Network

paper accepted at NIPS 2017 Bayesian Deep Learning workshop.


Games | Robotics | Finance |
- | - | - | 
<img src="img/alphago.png" alt="Drawing" style="width: 200px;"/> | <img src="img/robotics.png" alt="Drawing" style="width: 200px;"/> | <img src="img/finance.png" alt="Drawing" style="width: 200px;"/> 

### Markov Decision Process (MDP) and Reinforcement Learning (RL)

In Markov Decision Process (MDP), an agent is in state $s_t \in \mathcal{S}$, takes action $a_t \in \mathcal{A}$, then transitions to state $s_{t+1} \in \mathcal{S}$ and receives instant reward $r_t$. A mapping from state to action $\pi: \mathcal{S} \mapsto \mathcal{A}$ is a policy. The aim is to find policy to optimize cumulative reward
$$\mathbb{E}_\pi\big[ \sum_{t=0}^\infty r_t\gamma^t  \big]$$
where $\gamma$ is a discount factor. 

#### Action value function $Q^\pi(s,a)$
An agent starts with state action pair $s_0 = s,a_0 = a$ then follows policy $\pi$, then the action value function is defined as $$Q^\pi(s,a) = \mathbb{E}_\pi \big[ \sum_{t=0}^\infty r_t \gamma^t \big| s_0 = s, a_0 = a \big]$$

#### Bellman quation of optimality
The optimal policy $\pi^\ast$ has optimal action value function $Q^\ast(s,a)$ satisfy the following equality $$Q^\ast(s_t,a_t) = \max_a \mathbb{E}\big[ r_t + \gamma Q^\ast(s_{t+1},a) \big]$$
To obtain $\pi^\ast$ from $Q^\ast(s,a)$: $\pi^\ast(s) = \arg\max_a Q^\ast(s,a)$ (one-step lookahead).

#### Bellman error
One way to evaluate how suboptimal a policy is, is through Bellman error. Bellman error for state action pair $(s,a)$ under policy $\pi$ is $$(Q^\pi(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q^\pi(s_{t+1},a) \big])^2$$

Notice that optimal $Q^\ast(s,a)$ has exact zero Bellman error for all $(s,a)$. A policy $\pi$ with zero Bellman error is also optimal. Suboptimal policies tend to have small Bellman error (though not necessarily).


### Deep Q Network (DQN)
Deep Q Network is proposed in [Mnih, 2013], as one of the first successful deep RL framework to solve challenging control tasks. Consider parameterizing a neural network $Q_\theta(s,a)$ with parameter $\theta$ to approximate $Q^\ast(s,a)$. Use SGD to minimize the objective
$$\mathbb{E}_{\pi_\theta} \big[(Q_\theta(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a) \big])^2\big]$$

If $Q_\theta(s,a)$ minimizes the above objective then potentially $Q_\theta(s,a) \approx Q^\ast(s,a)$ and greedy policy w.r.t. $Q_\theta(s,a)$ is close to optimal.

### Exploration in RL
Exploration is essential in solving RL tasks. Being greedy usually gives suboptimal performance because the agent does not explore and cannot find potentially more optimal patterns. Efficient exploration scheme is critical in solving challenging control tasks.

### Variational Deep Q Network (VDQN)
Original DQN uses $\epsilon-$greedy exploration. Such exploration scheme is myopic and is not efficient enough in challenging tasks. Variational DQN maintains a distribution over DQN parameters $\theta \sim q_\phi(\theta)$ with parameter $\phi$.

Proposed objective: $$\mathbb{E}_{\theta\sim q_\phi(\theta)} \big[ \mathbb{E}_{\pi_\theta} \big[(Q_\theta(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a) \big])^2\big]  \big] - \lambda \mathbb{H}(q_\phi(\theta))$$

First term encourages minimization of expected Bellman error -- search for optimal policy; Second term encourages high entropy of distribution $q_\phi(\theta)$ -- encourage exploration. 

In practice, the 'regression target' $d_t = \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a)\big]$ is computed by a target network. Let targets $D = \{d_t\}$ be data. We can interpret $Q_\theta(s,a)$ as a Bayesian neural network with uniform improper prior $p(\theta) \propto 1$, the above objective reduces to 
$$\mathbb{KL}\big[q_\phi(\theta) \ || \ p(\theta|D)\big]$$
we can use Edward to update distribution parameter $\phi$.


In [1]:
# below are all necessary packages
import edward                # probabilistic programming package
import tensorflow as tf      # autodiff package
import chainer               # autodiff package
import gym                   # OpenAI gym for RL environments

## Experiments: VDQN architecture

For control tasks in current experiments, we use feed forward neural network. The input is state $s_t$ at each time step. The feed forward network has 2 hidden layers each with $100$ hidden units with relu activation. The output is a probability vector $p$ of choosing actions (discrete action space).

In all experiments, VDQN applies KLqp inference with $\lambda = 0.02$. MAP inference recovers conventional DQN. Any inference algorithm is approximate minimization of Bellman error. VDQN parameterizes the distribubtion as component-wise gaussian with mean and variance parameters.

## Testing Environment
### OpenAI gym control tasks
Learn policies to manipulate mechanical systems. 

MountainCar | CartPole | Acrobot |
- | - | - | 
<img src="img/mountaincarimg.png" alt="Drawing" style="width: 200px;"/> | <img src="img/cartpoleimg.jpg" alt="Drawing" style="width: 200px;"/> | <img src="img/acrobotimg.jpg" alt="Drawing" style="width: 200px;"/> 

### Chain MDP 
Learn policies to reach far-away destination without getting trapped in local optimality. Benchmark for exploration. Take exponential time to even reach the optimal state for random exploration strategy.

<img src="img/chainMDP.png" alt="Drawing" style="width: 500px;"/> 

## OpenAI gym control tasks

Compare DQN with VDQN. Both agents have the same architecture. DQN applies $\epsilon-$greedy exploration with $\epsilon = 0.1$. VDQN applies KLqp inference with $\lambda = 0.02$.

#### Observations
1. Both algorithms solve control tasks within given number of iterations
2. VDQN makes progress faster but tends to converge more slowly on a certain tasks. Potentially because the exploration prevents fast convergence to optimal policy.

CartPole-v0 | CartPole-v1 | MountainCar | Acrobot |
- | - | - | - | 
<img src="img/CartPole-v0.png" alt="Drawing" style="width: 200px;"/> | <img src="img/CartPole-v1.png" alt="Drawing" style="width: 200px;"/> | <img src="img/MountainCar-v0.png" alt="Drawing" style="width: 200px;"/> | <img src="img/Acrobot-v1.png" alt="Drawing" style="width: 200px;"/>

## Chain MDP

Compare DQN, VDQN and NoisyNet. All agents have the same architecture. DQN applies $\epsilon-$greedy exploration with $\epsilon = 0.1$. VDQN applies KLqp inference with $\lambda = 0.02$. VDQN and NoisyNet starts out with the same gaussian distribution parameters for each DQN parameter.

#### Observations
1. All algorihtms solve the small $N$ Chain MDP within given numbber of iterations, NoisyNet and VDQN have similar training curves.
2. For medium size $N$, DQN does not make progress. NoisyNet displays high variance in performance. VDQN solves the task successfully.
3. DQN and NoisyNet both get stuck. VDQN solves the task with higher variance.


$N=5$ | $N=50$ | $N=100$ | 
- | - | - | 
<img src="img/MDPN5_NoisyNet.png" alt="Drawing" style="width: 300px;"/> | <img src="img/MDPN50_NoisyNet.png" alt="Drawing" style="width: 300px;"/> | <img src="img/MDPN100_NoisyNet.png" alt="Drawing" style="width: 300px;"/> 


## Chain MDP: Measuring Exploration -- State Visitation Counts

Plot state visitation counts below to illustrate the exploration efficiency of three algorithmd. For a given state $s_i$, $1\leq i\leq N$, if it is ever visited in one episode we set $c_i = 1$. The plot is a moving average of $c_i$ over time, to approximate the probability of visiting state $i$ under current policy. You need to visit the states frequently enough to learn a good policy!

Illustrate three curves
1. $s_1$: locally optimal, see if the agent is stuck
2. $s_\frac{N}{2}$: see if the agent ever crosses $s_{N/2}$ to explore another half of the chain
3. $s_N$: global optimal, see if the agent explores consistently

#### Observations
1. DQN visits $s_1$ increasingly frequently and gets stuck at local optimum. Occasionally, DQN goes over $s_{\frac{N}{2}}$ (green spike) and does not have enough momentum ( consisntecy in exploration) to reach $s_N$.
2. NoisyNet has more consistency in exploration as illustrated in $N = 32$ case. For $N = 128$, however, the exploration is much slower (blue and green spike at the end show that NoisyNet could potentially solve the problem, while taking much more time)
3. VDQN explores the most consistently. VDQN maintains a non-trivial visitation probability from the start of training. It can then visit $s_N$ for sufficient number of times and quickly converge to optimal policy.


DQN $N = 32$ | NoisyNet $N = 32$ | VDQN $N = 32$ | 
- | - | - | 
<img src="img/DQN_MDPN32visit.png" alt="Drawing" style="width: 300px;"/> | <img src="img/NoisyNet_MDPN32visit.png" alt="Drawing" style="width: 300px;"/> | <img src="img/VIDQN_MDPN32visit.png" alt="Drawing" style="width: 300px;"/> 

DQN $N = 128$ | NoisyNet $N = 128$ | VDQN $N = 128$ | 
- | - | - | 
<img src="img/DQN_MDPN128visit.png" alt="Drawing" style="width: 300px;"/> | <img src="img/NoisyNet_MDPN100visit.png" alt="Drawing" style="width: 300px;"/> | <img src="img/VIDQN_MDPN128visit.png" alt="Drawing" style="width: 300px;"/> 




## Why a potentially better exploration scheme?

### Comparison across algorithms
1. DQN explores by randomization of single actions. Such exploration breaks temporal correlation in the current policy. Feedback not consistent.
2. NoisyNet explores by randomization in policy space. Feedback is consistent, exploration is more consistent. May get stuck due to premature convergence.
3. VDQN explicitly encourages high entropy in distribubtion over policies. This encourages the agent to achieve low expected Bellman error, while encompassing as many policies as possible -- consistent and persistent exploration. May slow down learning.

### Connection to Thompson sampling
Thompson sampling is a successful exploration scheme in multi-arm bandit and RL. The idea is to maintain a widely spread belief (prior) over MDPs. Take action and then updata belief (posterior). This strikes a nice balance between exploration vs. exploitation.

VDQN executes approximate Thompson sampling, approximation comes from
1. Posterior sampling is not exact, replaced by variational approximation
2. Target values are not from optimal value function, but from target network
